<a href="https://colab.research.google.com/github/Bossism/DailyStudy/blob/master/fakenews_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
cd ./drive/MyDrive/

/content/drive/MyDrive


In [20]:
!ls

'Colab Notebooks'


In [21]:
cd ./Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [23]:
!ls

 bert_classification.ipynb	     Untitled13.ipynb
 BertLeeMeng			     Untitled14.ipynb
 BertLeeMeng.zip		     Untitled15.ipynb
 convolution.ipynb		     Untitled16.ipynb
'crawler (1).ipynb'		     Untitled17.ipynb
 crawler.ipynb			     Untitled18.ipynb
 datawhalechina_hw2.ipynb	     Untitled19.ipynb
 FakeNewsClassification_Bert.ipynb   Untitled1.ipynb
 “hw7_bert”的副本		    'Untitled (2)'
 matplotlib.ipynb		     Untitled20.ipynb
 Multi_classification.ipynb	     Untitled2.ipynb
 PyTorchBasicKnowledge.ipynb	    'Untitled (3)'
 PyTorch.ipynb			     Untitled3.ipynb
 Regression.ipynb		     Untitled4.ipynb
 Seq2Seq_attention.ipynb	     Untitled5.ipynb
 Seq2Seq.ipynb			     Untitled6.ipynb
 transformer.ipynb		     Untitled7.ipynb
 Two_Classification.ipynb	     Untitled8.ipynb
 Untitled			     Untitled9.ipynb
 Untitled0.ipynb		     wuenda_ex1.ipynb
'Untitled (1)'			     基础知识.ipynb
 Untitled10.ipynb		     模型构建.ipynb
 Untitled11.ipynb		     泰坦尼克号生存率预测.ipynb
 Untitled12.ipynb


In [24]:
cd BertLeeMeng/data/

/content/drive/MyDrive/Colab Notebooks/BertLeeMeng/data


In [26]:
cd .

/content/drive/MyDrive/Colab Notebooks/BertLeeMeng/data


In [1]:
cd /content/drive/MyDrive/Colab Notebooks/BertLeeMeng/

/content/drive/MyDrive/Colab Notebooks/BertLeeMeng


In [31]:
!ls

bert_learning.py  data	main.py  test.tsv


In [ ]:
!python bert_learning.py

In [ ]:
pip install transformers

In [3]:
import torch
from transformers import BertTokenizer
from IPython.display import clear_output

PRETRAINED_MODEL_NAME = "bert-base-chinese"

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

clear_output()
# print("PyTorch 版本:", torch.__version__)  # PyTorch 版本: 1.9.1+cpu

vocab = tokenizer.vocab
# print("字典大小：", len(vocab))  # 21128
# tokenizer.vocab_size

import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]
print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids):
    print("{0:15}{1:10}".format(t, id))

text = "[CLS] 等到潮水[MASK]了, 就知道谁没穿裤子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(text)
print(tokens, '...')
print(ids, '...')
# print('qqq', 'hhh', 'xxx')

# 对于给定的[MASK]句子，bert会填入什么字
from transformers import BertForMaskedLM

tokens_tensor = torch.tensor([ids])
segments_tensor = torch.zeros_like(tokens_tensor)
maskedLM_model = BertForMaskedLM.from_pretrained(PRETRAINED_MODEL_NAME)
clear_output()
maskedLM_model.eval()
with torch.no_grad():
    outputs = maskedLM_model(tokens_tensor, segments_tensor)
    predictions = outputs[0]
del maskedLM_model
masked_index = 5
k = 3
probs, indices = torch.topk(torch.softmax(predictions[0, masked_index], -1), k)
predicted_tokens = tokenizer.convert_ids_to_tokens(indices.tolist())
# 展示top k 个可能的字， 一般取top 1作为预测值
print("輸入 tokens ：", tokens[:10], '...')
print('-' * 50)
for i, (t, p) in enumerate(zip(predicted_tokens, probs), 1):
    tokens[masked_index] = t
    print("Top {} ({:2}%)：{}".format(i, int(p.item() * 100), tokens[:10]), '...')

# 用 Bert fine tune下游任务
# 1. 准备原始文本数据
import pandas as pd

# 簡單的數據清理，去除空白標題的 examples
df_train = pd.read_csv("./data/train.csv")
empty_title = ((df_train['title2_zh'].isnull()) \
               | (df_train['title1_zh'].isnull()) \
               | (df_train['title2_zh'] == '') \
               | (df_train['title2_zh'] == '0'))
df_train = df_train[~empty_title]

# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 30
df_train = df_train[~(df_train.title1_zh.apply(lambda x : len(x)) > MAX_LENGTH)]
df_train = df_train[~(df_train.title2_zh.apply(lambda x : len(x)) > MAX_LENGTH)]

# 只用 1% 訓練數據看看 BERT 對少量標註數據有多少幫助
SAMPLE_FRAC = 0.01
df_train = df_train.sample(frac=SAMPLE_FRAC, random_state=9527)

# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['title1_zh', 'title2_zh', 'label']]
df_train.columns = ['text_a', 'text_b', 'label']

# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
df_train.to_csv("./data/train.tsv", sep="\t", index=False)

print("訓練樣本數：", len(df_train))
df_train.head()
df_train.label.value_counts() / len(df_train)

df_test = pd.read_csv("./data/test.csv")
df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]
df_test.columns = ["text_a", "text_b", "id"]
df_test.to_csv("./data/test.tsv", sep='\t', index=False)
print("预测样本数：", len(df_test))
df_test.head()

# 2. 将原始数据转化为BERT兼容的输入格式
from torch.utils.data import Dataset
class FakeNewsDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]
        self.mode = mode
        self.df = pd.read_csv("./data/" + mode + ".tsv", sep="\t").fillna("")
        self.len = len(self.df)
        self.label_map = {'agreed': 0, 'disagreed': 1, 'unrelated': 2}
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        if self.mode == "test":
            text_a, text_b = self.df.iloc[idx, :2].values
            label_tensor = None
        else:
            text_a, text_b, label = self.df.iloc[idx, :].values
            label_id = self.label_map[label]
            label_tensor = torch.tensor(label_id)

        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a + ["[SEP]"]
        len_a = len(word_pieces)

        tokens_b = self.tokenizer.tokenize(text_b)
        word_pieces += tokens_b + ["[SEP]"]
        len_b = len(word_pieces) - len_a

        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        segments_tensor = torch.tensor([0] * len_a + [1] * len_b, dtype=torch.long)

        return (tokens_tensor, segments_tensor, label_tensor)

    def __len__(self):
        return self.len

trainset = FakeNewsDataset("train", tokenizer=tokenizer)
sample_idx = 0
text_a, text_b, label = trainset.df.iloc[sample_idx].values
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)
print(f"""[原始文本]
句子 1：{text_a}
句子 2：{text_b}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")


"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `FakeNewsDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]

    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None

    # zero padding 到同一序列长度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, batch_first=True)

    # attention masks 将tokens_tensors里头不为zero padding 的位置设为1， 让BERT只关注这些位置的tokens
    maskes_tensors = torch.zeros(tokens_tensors.shape, dtype=torch.long)
    maskes_tensors = maskes_tensors.masked_fill(tokens_tensors != 0, 1)

    return tokens_tensors, segments_tensors, maskes_tensors, label_ids

BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, collate_fn=create_mini_batch)

data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data
print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")

# 3. 在BERT之上加入新layer成下游任务模型
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-chinese"
NUM_LABELS = 3

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)

clear_output()

# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0

    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]

            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors,
                            token_type_ids=segments_tensors,
                            attention_mask=masks_tensors)

            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)

            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()

            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))

    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)


def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]

model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")

# 4. 训练该下游人物模型
# 訓練模式
model.train()
# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 6  # 幸運數字
for epoch in range(EPOCHS):

    running_loss = 0.0
    for data in trainloader:
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=tokens_tensors,
                        token_type_ids=segments_tensors,
                        attention_mask=masks_tensors,
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()

    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    torch.save(model, "model_{}.pth".format(epoch + 1))

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=3, bias=True)
device: cuda:0
classification acc: 0.4681972149040271

整個分類模型的參數量：102269955
線性分類器的參數量：2307

[epoch 1] loss: 25.672, acc: 0.834
[epoch 2] loss: 18.412, acc: 0.889
[epoch 3] loss: 14.356, acc: 0.923
[epoch 4] loss: 10.261, acc: 0.936
[epoch 5] loss: 7.628, acc: 0.960
[epoch 6] loss: 5.912, acc: 0.967


In [5]:
df_test = pd.read_csv("./data/test.csv")
df_test = df_test.loc[:, ["title1_zh", "title2_zh", "id"]]
df_test.columns = ["text_a", "text_b", "id"]
df_test.to_csv("./data/test.tsv", sep='\t', index=False)
print("预测样本数：", len(df_test))
df_test.head()

预测样本数： 80126


,text_a,text_b,id
0,萨拉赫人气爆棚!埃及总统大选未参选获百万选票 现任总统压力山大,辟谣！里昂官方否认费基尔加盟利物浦，难道是价格没谈拢？,321187
1,萨达姆被捕后告诫美国的一句话，发人深思,10大最让美国人相信的荒诞谣言，如蜥蜴人掌控着美国,321190
2,萨达姆此项计划没有此国破坏的话，美国还会对伊拉克发动战争吗,萨达姆被捕后告诫美国的一句话，发人深思,321189
3,萨达姆被捕后告诫美国的一句话，发人深思,被绞刑处死的萨达姆是替身？他的此男人举动击破替身谣言！,321193
4,萨达姆被捕后告诫美国的一句话，发人深思,中国川贝枇杷膏在美国受到热捧？纯属谣言！,321191


In [7]:
# 5. 对新样本做推论
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = FakeNewsDataset("test", tokenizer=tokenizer)
testloader = DataLoader(testset, batch_size=256,
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
torch.load("model_5.pth")
predictions = get_predictions(model, testloader)

# 用來將預測的 label id 轉回 label 文字
index_map = {v: k for k, v in testset.label_map.items()}

# 生成 Kaggle 繳交檔案
df = pd.DataFrame({"Category": predictions.tolist()})
df['Category'] = df.Category.apply(lambda x: index_map[x])
df_pred = pd.concat([testset.df.loc[:, ["id"]],
                          df.loc[:, 'Category']], axis=1)
df_pred.to_csv('bert_1_prec_training_samples.csv', index=False)
df_pred.head()

,id,Category
0,321187,unrelated
1,321190,unrelated
2,321189,unrelated
3,321193,unrelated
4,321191,unrelated
